# Q4: Study Neural Collapse in the presence of OOD data

This notebook studies how Neural Collapse metrics evolve when Out-of-Distribution (OOD) data
is treated as an extra class alongside the ID classes.

We focus on:
- **NC5** — ID/OOD Orthogonality (eq. 5 of the NECO paper)
- **NC1** (ID only vs ID+OOD) — Does OOD break within/between-class collapse?
- **NC2** (ID only vs ID+OOD) — Does OOD break equiangularity?

Reference:
> Ben Ammar et al., *"NECO: Neural Collapse Based Out-of-Distribution Detection"*, ICLR 2024.

## Setup

In [ ]:
# Mount Google Drive (for saving checkpoints)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository
import os
if not os.path.exists('/content/OOD-Detection-Project---CSC_5IA23'):
    !git clone https://github.com/DiegoFleury/OOD-Detection-Project---CSC_5IA23/

!git stash
!git checkout contente
!git pull
%cd /content/OOD-Detection-Project---CSC_5IA23

In [ ]:
# Install dependencies
!pip install -q torch torchvision matplotlib seaborn scikit-learn pyyaml imageio tqdm

In [ ]:
# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml
import glob
import re
import os

from src.models import ResNet18
from src.data import get_cifar100_loaders

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Load config
with open('configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration:")
print(yaml.dump(config, default_flow_style=False))

## 1. Load Data

- **CIFAR-100** — In-Distribution (ID)
- **SVHN** and **CIFAR-10** — Out-of-Distribution (OOD)

In [ ]:
# ID data: CIFAR-100
print("Loading CIFAR-100 (ID) dataset...")

train_loader, val_loader, test_loader = get_cifar100_loaders(
    data_dir=config['data']['data_dir'],
    batch_size=config['training']['batch_size'],
    num_workers=config['data']['num_workers'],
    augment=False,
    val_split=config['training']['val_split']
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches:   {len(val_loader)}")
print(f"Test batches:  {len(test_loader)}")

In [ ]:
# OOD data: SVHN + CIFAR-10
import torchvision
import torchvision.transforms as transforms

ood_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    ),
])

# SVHN
print("Loading SVHN (OOD)...")
svhn_dataset = torchvision.datasets.SVHN(
    root=config['data']['data_dir'], split='test',
    transform=ood_transform, download=True,
)
svhn_loader = torch.utils.data.DataLoader(
    svhn_dataset, batch_size=config['training']['batch_size'],
    shuffle=False, num_workers=config['data']['num_workers'],
)
print(f"SVHN test samples: {len(svhn_dataset)}")

# CIFAR-10
print("Loading CIFAR-10 (OOD)...")
cifar10_dataset = torchvision.datasets.CIFAR10(
    root=config['data']['data_dir'], train=False,
    transform=ood_transform, download=True,
)
cifar10_loader = torch.utils.data.DataLoader(
    cifar10_dataset, batch_size=config['training']['batch_size'],
    shuffle=False, num_workers=config['data']['num_workers'],
)
print(f"CIFAR-10 test samples: {len(cifar10_dataset)}")

In [ ]:
# Visualize OOD samples
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
fig.suptitle("OOD Samples", fontsize=14, y=1.02)

svhn_images, _ = next(iter(svhn_loader))
c10_images, _ = next(iter(cifar10_loader))

for i in range(8):
    for row, (imgs, name) in enumerate([(svhn_images, 'SVHN'), (c10_images, 'CIFAR-10')]):
        img = imgs[i].permute(1, 2, 0).numpy()
        img = img * np.array([0.2675, 0.2565, 0.2761]) + np.array([0.5071, 0.4867, 0.4408])
        img = np.clip(img, 0, 1)
        axes[row, i].imshow(img)
        axes[row, i].axis('off')
        if i == 0:
            axes[row, i].set_ylabel(name, fontsize=12, rotation=0, labelpad=50)

plt.tight_layout()

ood_fig_dir = os.path.join(config['paths']['figures'], 'ood')
os.makedirs(ood_fig_dir, exist_ok=True)
plt.savefig(os.path.join(ood_fig_dir, 'ood_samples.png'), dpi=150, bbox_inches='tight')
plt.show()

## 2. Import NC OOD Module

In [ ]:
from src.neural_collapse.nc_ood import (
    load_checkpoints_and_analyze_ood,
    plot_nc5_convergence,
    plot_ood_summary,
    save_ood_metrics_yaml,
    NCOODTracker,
)

print("✅ NC OOD module imported successfully!")

## 3. NC5 Analysis Across Training Epochs

NC5 (eq. 5 of the paper) measures whether ID class means become orthogonal to the OOD global mean during training:

$$\text{OrthoDev} = \text{Avg}_c \left| \frac{\langle \mu_c, \mu_G^{\text{OOD}} \rangle}{\|\mu_c\| \cdot \|\mu_G^{\text{OOD}}\|} \right|$$

This should decrease toward 0 as the model enters the Terminal Phase of Training.

In [ ]:
# Configure paths
checkpoint_dir = config['paths']['checkpoints']
figures_dir = os.path.join(config['paths']['figures'], 'ood')
metrics_dir = config['paths']['metrics']
os.makedirs(figures_dir, exist_ok=True)
os.makedirs(metrics_dir, exist_ok=True)

# Verify checkpoints
def get_epoch_num(path):
    match = re.search(r'epoch(\d+)', path)
    return int(match.group(1)) if match else 0

checkpoints = glob.glob(os.path.join(checkpoint_dir, 'resnet18_cifar100_*.pth'))
print(f"📁 Checkpoints found: {len(checkpoints)}")
for ckpt in sorted(checkpoints, key=get_epoch_num)[:5]:
    print(f"   - {os.path.basename(ckpt)}")
if len(checkpoints) > 5:
    print(f"   ... and {len(checkpoints) - 5} more")

In [ ]:
# NC5 analysis with SVHN as OOD
print("🔬 Analyzing NC metrics with OOD=SVHN...")
print(f"   Device: {device}")
print(f"   ID classes: {config['model']['num_classes']}")
print()

tracker_svhn = load_checkpoints_and_analyze_ood(
    checkpoint_dir=checkpoint_dir,
    model_class=ResNet18,
    id_loader=train_loader,
    ood_loader=svhn_loader,
    device=device,
    num_classes=config['model']['num_classes'],
    checkpoint_pattern='resnet18_cifar100_*.pth',
    epoch_regex=r'epoch(\d+)',
    verbose=True,
)

print("\n" + tracker_svhn.summary())

In [ ]:
# NC5 analysis with CIFAR-10 as OOD
print("🔬 Analyzing NC metrics with OOD=CIFAR-10...")
print()

tracker_cifar10 = load_checkpoints_and_analyze_ood(
    checkpoint_dir=checkpoint_dir,
    model_class=ResNet18,
    id_loader=train_loader,
    ood_loader=cifar10_loader,
    device=device,
    num_classes=config['model']['num_classes'],
    checkpoint_pattern='resnet18_cifar100_*.pth',
    epoch_regex=r'epoch(\d+)',
    verbose=True,
)

print("\n" + tracker_cifar10.summary())

## 4. Visualize Results

### 4.1 NC5 Convergence

Reproducing Figure 1 (right) and Figure D.11 from the NECO paper.

In [ ]:
# Individual NC5 plots
fig_svhn = plot_nc5_convergence(
    tracker_svhn, ood_name="SVHN", id_name="CIFAR-100",
    save_dir=figures_dir,
)
plt.show()

fig_c10 = plot_nc5_convergence(
    tracker_cifar10, ood_name="CIFAR-10", id_name="CIFAR-100",
    save_dir=figures_dir,
)
plt.show()

In [ ]:
# Combined NC5 plot (both OOD datasets)
fig, ax = plt.subplots(figsize=(9, 6))

ax.plot(
    tracker_svhn.epochs, tracker_svhn.nc5_orthodev,
    'b-o', markersize=4, linewidth=2,
    label='ID (CIFAR-100) / OOD (SVHN)',
)
ax.plot(
    tracker_cifar10.epochs, tracker_cifar10.nc5_orthodev,
    'g-s', markersize=4, linewidth=2,
    label='ID (CIFAR-100) / OOD (CIFAR-10)',
)

ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('NC5: ID/OOD OrthoDev', fontsize=12)
ax.set_title('NC5: Convergence to ID/OOD Orthogonality (ResNet-18)', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
fig.tight_layout()

plt.savefig(os.path.join(figures_dir, 'nc5_combined.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"Final NC5 OrthoDev (SVHN):     {tracker_svhn.nc5_orthodev[-1]:.6f}")
print(f"Final NC5 OrthoDev (CIFAR-10): {tracker_cifar10.nc5_orthodev[-1]:.6f}")

### 4.2 Full NC Summary with OOD as Extra Class

Following Appendix D.2 of the paper: compare NC1 and NC2 on ID data only vs. ID+OOD (treating OOD as class C+1).

In [ ]:
fig_summary_svhn = plot_ood_summary(
    tracker_svhn, ood_name="SVHN", save_dir=figures_dir,
)
plt.show()

fig_summary_c10 = plot_ood_summary(
    tracker_cifar10, ood_name="CIFAR-10", save_dir=figures_dir,
)
plt.show()

### 4.3 Interpretation

We analyse the key findings from the NC5 + NC1/NC2 evolution.

In [ ]:
print("\n" + "=" * 60)
print("ANALYSIS: NC5 & OOD BEHAVIOUR")
print("=" * 60)

for name, tracker in [('SVHN', tracker_svhn), ('CIFAR-10', tracker_cifar10)]:
    print(f"\n--- OOD = {name} ---")

    # NC5 decrease
    nc5_start = tracker.nc5_orthodev[0]
    nc5_end = tracker.nc5_orthodev[-1]
    nc5_reduction = (nc5_start - nc5_end) / nc5_start * 100
    print(f"  NC5: {nc5_start:.4f} → {nc5_end:.4f}  ({nc5_reduction:+.1f}%)")

    # NC1 comparison
    nc1_id = tracker.Sw_invSb_id_only[-1]
    nc1_ood = tracker.Sw_invSb_id_ood[-1]
    print(f"  NC1 (ID only):  {nc1_id:.4f}")
    print(f"  NC1 (ID+OOD):   {nc1_ood:.4f}")
    if nc1_ood > nc1_id:
        print(f"  → OOD increases NC1 by {(nc1_ood/nc1_id - 1)*100:.1f}% (breaks collapse)")

    # NC2 comparison
    eq_id = tracker.nc2_equiang_id_only[-1]
    eq_ood = tracker.nc2_equiang_id_ood[-1]
    print(f"  NC2 equiang (ID only):  {eq_id:.4f}")
    print(f"  NC2 equiang (ID+OOD):   {eq_ood:.4f}")
    if eq_ood > eq_id:
        print(f"  → OOD breaks equiangularity (+{(eq_ood/eq_id - 1)*100:.1f}%)")

print("\n" + "=" * 60)

## 5. Save Metrics

In [ ]:
save_ood_metrics_yaml(
    tracker_svhn,
    os.path.join(metrics_dir, 'nc_ood_svhn_metrics.yaml'),
)
save_ood_metrics_yaml(
    tracker_cifar10,
    os.path.join(metrics_dir, 'nc_ood_cifar10_metrics.yaml'),
)

# Also save as JSON
import json

for name, tracker in [('svhn', tracker_svhn), ('cifar10', tracker_cifar10)]:
    data = tracker.to_dict()
    for key, values in data.items():
        if isinstance(values, list):
            data[key] = [
                v if not (isinstance(v, float) and np.isnan(v)) else None
                for v in values
            ]
    json_path = os.path.join(metrics_dir, f'nc_ood_{name}_metrics.json')
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"💾 Saved: {json_path}")

## 6. Final Summary

In [ ]:
print("\n" + "=" * 60)
print("NC OOD ANALYSIS SUMMARY")
print("=" * 60)

print(f"\nModel: ResNet-18 trained on CIFAR-100")
print(f"Epochs analyzed: {len(tracker_svhn.epochs)}")
print(f"Epoch range: {tracker_svhn.epochs[0]} → {tracker_svhn.epochs[-1]}")

print(f"\n--- Final NC5 OrthoDev ---")
print(f"  SVHN:     {tracker_svhn.nc5_orthodev[-1]:.6f}")
print(f"  CIFAR-10: {tracker_cifar10.nc5_orthodev[-1]:.6f}")

print(f"\n--- Files Saved ---")
print(f"  Figures: {figures_dir}/")
print(f"  Metrics: {metrics_dir}/")

print("\n" + "=" * 60)

## 7. Commit Results to GitHub

In [ ]:
# !git add results/figures/ood/
# !git add results/metrics/
# !git commit -m "Add Q4 NC OOD analysis: NC5 convergence + NC1/NC2 with OOD"
# !git push
#
# print("Results committed to GitHub!")